In [10]:
#| default_exp ors

In [1]:
#| export
import numpy as np

In [5]:
#| export
def generate_img_occlusion_list_pos_neg_saliency(img, saliency, patch_size, saliency_reduce_func=np.mean):
    """
    Generates a list of img that has been sequentially occluded according to the patch_size,
    whereby `stride=patch_size`.
    The function modified for use with occlusion run sensitivity metric, and, therefore, is not meant
    to be used outside of said scope
    
    input :
      - img : ndarray (Image to create occluded version of)
      - patch_size : int (Size of the occluded patch)
      - saliency : ndarray (optional saliency map to generate a reduced saliency value for each occluded region)
      - saliency_reduce_func : function (optional iterative-wise function to reduce saliency value into a value)
      
    return :
      - occlusion_img_list : ndarray (Array of occluded versions of the image)
      - pos_saliency_statistic : ndarray (Array of reduce saliency value of positive phase)
      - neg_saliency_statistic : ndarray (Array of reduce saliency value of negative phase)
    """
    occlusion_img_list = []
    pos_saliency_statistic = []
    neg_saliency_statistic = []
    saliency = np.array(Image.fromarray(saliency).resize((img.shape[0], img.shape[1])))
    
    for top_left_y in range(0, img.shape[1], patch_size):
        for top_left_x in range(0, img.shape[0], patch_size):
            patched_image = apply_patch(img, top_left_x, top_left_y, patch_size).astype('float32') / 255.0
            occlusion_img_list.append(patched_image)
            fil_sal = saliency[top_left_y : top_left_y+patch_size, top_left_x : top_left_x+patch_size]
            pos_saliency_statistic.append(saliency_reduce_func(fil_sal))
            neg_saliency_statistic.append(saliency_reduce_func(1 - fil_sal))
            
    return np.array(occlusion_img_list), np.array(pos_saliency_statistic), np.array(neg_saliency_statistic)

In [6]:
#| export
def occlusion_run_accu_sensitivity(original_image, model_original_mask, model_original_idx, model_original_probs,
                                   patch_size, model_func, saliency_reduce_func=np.mean):
    """
    Calculate the occlusion-run sensitivity metric based on the accuracy of the model.
    The metric consists of two phases and the final result is a calculated using both phases.
    For more details, refer to the documentation.
    
    input :
      - original_image : ndarray (The original un-occluded image)
      - model_original_mask : ndarray (Saliency of the un-occluded image as computed through a saliency generation proces)
      - model_original_idx : int (The original label of the un-occluded image as give by the model)
      - model_original_probs : ndarray (Probability of the un-occluded image as computed by the model)
      - patch_size : int (Size of the occlusion patch)
      - model_func : function (Function for the model; the model must return probability as its third and last return)
      - saliency_reduce_func : function (Function used to reduce saliency values into a scalara)
      
    return :
      - score : float32 (Final score for the occlusion run sensitivity)
      - pos_tuple : (x_deltas : ndarray, scaled_occluded_metric : ndarray, occluded_metric : ndarray) (Value for the positive phase)
      - neg_tuple : (x_deltas : ndarray, scaled_occluded_metric : ndarray, occluded_metric : ndarray) (Value for the negative phase)
    """
    gen_occlu_images, pos_saliency_stat, neg_saliency_stat = generate_img_occlusion_list_pos_neg_saliency(original_image, model_original_mask,
                                                                                                          patch_size, saliency_reduce_func)
    # Positive Saliency
    pos_saliency_stat = pos_saliency_stat**2
    pos_saliency_stat = pos_saliency_stat / pos_saliency_stat.sum()
    pos_x_deltas = (pos_saliency_stat*100).astype(np.int32)
    
    occluded_run_metric = np.array([])
    for occu_img in gen_occlu_images:
        _, _, model_probs = model_func(occu_img)
        occluded_run_metric = np.append(occluded_run_metric, model_probs[model_original_idx])
    
    pos_scaled_occluded_run_metric = np.repeat(occluded_run_metric, pos_x_deltas)
    
    pos_original_area = model_original_probs[model_original_idx] * np.sum(pos_x_deltas)
    pos_occu_area_integral = []
    for idx in range(1, len(pos_scaled_occluded_run_metric)):
        x_delta = 1.0
        h = np.absolute(pos_scaled_occluded_run_metric[idx] - pos_scaled_occluded_run_metric[idx-1])
        S = np.minimum(pos_scaled_occluded_run_metric[idx], pos_scaled_occluded_run_metric[idx-1])
        area = (S + (h/2)) * x_delta
        pos_occu_area_integral.append(area)
        
    pos_accu_area = sum(pos_occu_area_integral)
        
    # Negative Saliency
    neg_saliency_stat = neg_saliency_stat**2
    neg_saliency_stat = neg_saliency_stat / neg_saliency_stat.sum()
    neg_x_deltas = (neg_saliency_stat*100).astype(np.int32)
    
    neg_scaled_occluded_run_metric = np.repeat(occluded_run_metric, neg_x_deltas)
    
    neg_original_area = model_original_probs[model_original_idx] * np.sum(neg_x_deltas)
    neg_occu_area_integral = []
    for idx in range(1, len(neg_scaled_occluded_run_metric)):
        x_delta = 1.0
        h = np.absolute(neg_scaled_occluded_run_metric[idx] - neg_scaled_occluded_run_metric[idx-1])
        S = np.minimum(neg_scaled_occluded_run_metric[idx], neg_scaled_occluded_run_metric[idx-1])
        area = (S + (h/2)) * x_delta
        neg_occu_area_integral.append(area)
        
    neg_accu_area = sum(neg_occu_area_integral)
    neg_accu_area = np.clip(neg_original_area - neg_accu_area, 0.0, neg_original_area)
    
    # score = np.clip(1-(sum(occu_area_integral)/original_area), 0, 1.0)
    orig_total_area = pos_original_area + neg_original_area
    score = np.clip(1 - ((pos_accu_area + neg_accu_area) / orig_total_area), 0, 1.0)
    
    return score, (pos_x_deltas, pos_scaled_occluded_run_metric, occluded_run_metric), (neg_x_deltas, neg_scaled_occluded_run_metric, occluded_run_metric)

In [7]:
#| export
def plot_multiple_occlusion_sensitivity(axes, img, original_mask, original_probs, original_idx, model_func,
                                        classes, patch_sizes, saliency_reduce_func, pos_vs_neg_idx):
    """
    Visualize the occlusion-run sensitivity metric for an image.
    
    input :
      - axes : ndarray (Array of matplotlib axes of shape 3)
      - img : ndarray (The original un-occluded image)
      - original_mask : ndarray (Saliency of the un-occluded image as computed through a saliency generation proces)
      - original_idx : int (The original label of the un-occluded image as give by the model)
      - original_probs : ndarray (Probability of the un-occluded image as computed by the model)
      - patch_size : int (Size of the occlusion patch)
      - model_func : function (Function for the model; the model must return probability as its third and last return)
      - saliency_reduce_func : function (Function used to reduce saliency values into a scalara)
      - classes : list (ImageNet 1000 classes string list)
      - pos_vs_neg_idx : int (If 0 then visualize the positive phase and if 1 visualize the negative phase)
    """
    # img = Image.fromarray((img*255).astype(np.uint8))
    visualize_saliency_overlay(axes[0], Image.fromarray(img), original_mask, np.expand_dims(original_probs, 0), original_idx, classes);
    axes[0].get_xaxis().set_visible(False)
    axes[0].get_yaxis().set_visible(False)
    for idx, patch_size in enumerate(patch_sizes):
        score, *values = occlusion_run_accu_sensitivity_v2(img, original_mask, original_idx, original_probs, patch_size, model_func, saliency_reduce_func)
        (x_deltas, scaled_occluded_run_metric, occluded_run_metric) = values[pos_vs_neg_idx]
        x = np.arange(0, scaled_occluded_run_metric.shape[0], 1)
        y1 = np.repeat(original_probs[original_idx], scaled_occluded_run_metric.shape[0])
        y2 = scaled_occluded_run_metric
        
        if pos_vs_neg_idx == 1:
            axes[1+idx].fill_between(x, y2, original_probs[original_idx], where=(y2 < original_probs[original_idx]) , alpha=0.3, color='red', interpolate=True)
        else:
            axes[1+idx].fill_between(x, y2, where=(y2 < original_probs[original_idx]), alpha=0.3, color='red', interpolate=True)
        # plt.fill_between(x, y2, y1, where=(y1 < y2), alpha=0.3, color='green', interpolate=True)
        axes[1+idx].set_ylim(0, 1)
        axes[1+idx].plot(y1)
        axes[1+idx].plot(y2)
        axes[1+idx].text(0, 0+0.07, f"Total area displacement={score:.3f}", fontsize=8, color="white", bbox={'facecolor' : "red", "alpha" : 0.5, "pad" : 5})
        axes[1+idx].set_title(f"Area for patch size {patch_size}");

In [12]:
#| hide
from nbdev.export import nb_export
nb_export('ors.ipynb', '../../commons/api/metric')